# AI Community @ Семинар № 8, весна 2018
## Style transfer.

Обратимся к предыдущей тетрадке. В ней мы пытались восстановить текстуру из данного распредления. При этом мы пользовались $L = \sum_l w_{style,l} E_{style, l}$. Изменим задачу. Нам так же дают текстуру, но еще добавляют другое изображение $O$. Наша цель перенести стиль изображения-текстуры на изображение О.

Пример:
<img src='images/starry_night_transfer.png'>

Так же изменим функцию ошибки, а именно добавим часть, которая будут отвечать за оригинал изображения:
$$E_{content,l} = w_{content} \sum_l (F_{content}^l - F_{gen}^l)^2$$, где $F_{content}^l$ - сохраненная карта признаков оригинального изображения $O$ на слое $l$, $F_{gen}^l$ - карта признаков генерируемого изображения на слое $l$, $w_{content}$ - вес изображения $O$ в финальном лоссе. Тогда:
$$L = \sum_l w_{style,l} E_{style, l} + E_{content,l}$$
Это значение мы и будем оптимизировать.